<a href="https://colab.research.google.com/github/Muhammad-Saad-2/LangLearn/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#installing encessary libraries

%pip install langchain -q -U
%pip install langchain-community -q -U
%pip install langchain-huggingface -q -U
%pip install langchain_pinecone -q -U
%pip install sentence-transformers -q -U
%pip install langchain-google-genai -q -U
%pip install langchain-pinecone -q -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/

In [40]:
import getpass
import os
import time
from google.colab import userdata
from pinecone import Pinecone

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc =  Pinecone(api_key=pinecone_api_key) #create instance from the pinecone class

In [46]:
import time

index_name = "ragexplore"

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
index = pc.Index(index_name)


In [53]:
print(pc.describe_index(index_name))

{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'ragexplore-q2q9tvj.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'ragexplore',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [7]:
#text embedding instance

from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore

embedding = SentenceTransformer( "sentence-transformers/all-mpnet-base-v2")

def embed_function(texts):
  if isinstance(texts, str):
    return embedding.encode(texts)
  return [embedding.encode(text) for text in texts]


vector_store = PineconeVectorStore(
    index=index,
    embedding=embedding,
    text_key="text"

  )


In [48]:
#creating dummy docuemnts

from langchain_core.documents import Document
from uuid import uuid4
from sentence_transformers import SentenceTransformer



document_1 = Document(
    page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
)
document_2 = Document(
    page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
)
document_3 = Document(
    page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata={"year": 200, "director": "Satoshi Kon", "rating": 8.6},
)
document_4 = Document(
    page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
    metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
)
document_5 = Document(
    page_content="Toys come alive and have a blast doing so",
    metadata={"year": 1995, "genre": "animated"},
)
document_6 = Document(
    page_content="Three men walk into the Zone, three men walk out of the Zone",
    metadata={"year": 19, "rating": 8.3},
)
document_7 = Document(
    page_content="In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization",
    metadata={"year": 2004, "director": "Steven Spielberg", "rating": 8.6},
)
document_8 = Document(
    page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
    metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
)
document_9 = Document(
    page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
    metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
)
document_10 = Document(
    page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
    metadata={"year": 200, "director": "Satoshi Kon", "rating": 8.6},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10
  ]

#assigning uuids to each document
uuids = [str(uuid4()) for doc in range (0, len(documents))]



In [50]:
#importing the embedding model
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(model_name)

embeddings = []

for i, document in enumerate(documents):
  embedding = model.encode(document.page_content)
  embeddings.append(embedding)
  index.upsert(
      vectors=[(uuids[i], embedding.tolist(), document.metadata)],
      namespace="namespcae_1"
  )

In [ ]:
print(embedding)

In [52]:


import os
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI

# Replace with your actual API key
google_api_key = userdata.get('GOOGLE_API_KEY')

# Initialize the Gemini model, using 'gemini-pro' as the model family
# Valid options for model include: 'gemini-pro', 'gemini-turbo', 'text-bison@001'
llm = GoogleGenerativeAI(model="gemini-pro", api_key=google_api_key, temperature=0.7)  # Adjust temperature as needed

# Example usage with the documents and embeddings
generated_text = llm.invoke(
    "Generate a creative text that incorporates information from these movie summaries:\n\n"
    + "\n\n".join([doc.page_content for doc in documents])
    + "\n\nThe new text should be a cohesive story, not just a summary."
)
generated_text

"In a realm where reality and dreams intertwined, a team of audacious scientists dared to resurrect the colossal creatures of the past. Chaos erupted as dinosaurs rampaged through the modern world.\n\nMeanwhile, amidst the labyrinthine corridors of the human mind, Leo DiCaprio found himself trapped in an endless spiral of dreams. Each layer peeled back, revealing a deeper and more enigmatic world, the boundaries between reality and fantasy blurring.\n\nA brilliant psychologist, lured into the depths of the subconscious, navigated a treacherous maze of dreams within dreams within dreams. The line between inception and the reality of his own mind became dangerously thin.\n\nIn a quaint town, wholesome women radiated an aura of tranquility, their simple lives a stark contrast to the turmoil unfolding elsewhere. Yet, their charm proved irresistible to a group of men who yearned for their company.\n\nAs the sun dipped below the horizon, toys sprang to life, their laughter echoing through th